###### Nom : El Maadoudi
###### Prénom : Mehdi
###### Matricule : 21359
###### Section : 4MIN
# Remise Projet PageRank

On calcule PageRank avec la méthode de puissance :

$$p^{(k+1)} = T \cdot p^{(k)}$$

Principe :

- T[i, j] = probabilité d'aller de la page j vers la page i (colonne = page de départ)
- Si une page n'a aucun lien sortant (impasse) : prochaine page uniforme (1/N)
- Arrêt quand ça converge

# Imports

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

## 1) Lire les CSV
- names.csv : colonne Name (noms de pages)
- edges.csv : colonnes FromNode, ToNode (liens entre pages)


In [2]:
liste_pages = pd.read_csv("names.csv")     # Name
liste_liens = pd.read_csv("edges.csv")       # FromNode, ToNode

nombre_pages = len(liste_pages)

pages_depart = liste_liens["FromNode"].to_numpy() - 1   # j
pages_arrivee = liste_liens["ToNode"].to_numpy() - 1     # i

nombre_pages, len(liste_liens)


(199903, 10722190)

## 2) Liens sortants + impasses

In [3]:
nbr_liens_page = np.bincount(pages_depart, minlength=nombre_pages)
impasses = (nbr_liens_page == 0)

print(f"Nombre de pages sans liens sortants (impasses) : {impasses.sum()}")

Nombre de pages sans liens sortants (impasses) : 0


## 3) Construction de la matrice de transition $T$

Le dataset étant lourd, on utilise une **matrice creuse** de SciPy pour optimiser.

* **Format utilisé** : `csr_matrix` (Compressed Sparse Row), idéal pour les multiplications matrice-vecteur.
* **Contenu de $T$** : Si un lien existe de la page $j$ vers la page $i$, l'élément $T_{ij}$ reçoit la probabilité $$\frac{1}{nbr\_liens\_page[j]}$$
* **Gestion des impasses** : Les pages sans liens sortants ne sont pas stockées dans la matrice creuse pour économiser de l'espace. Leur poids est redistribué dynamiquement lors de l'itération pour simuler un saut aléatoire et de probabilité égale vers n'importe quelle page du réseau.

In [4]:
# Construction de la partie "liens" de la matrice T
# Chaque lien j -> i reçoit une probabilité de 1 / nbr_liens_page[j]
proba_liens_page = 1.0 / nbr_liens_page[pages_depart]
T = csr_matrix((proba_liens_page, (pages_arrivee, pages_depart)), shape=(nombre_pages, nombre_pages))

## 4) Méthode de puissance avec facteur d'amortissement ($d$)

L'objectif est de calculer le vecteur de PageRank $\vec{p}^{(\infty)}$ par itérations successives. Pour éviter les boucles, on utilise un **facteur d'amortissement $d$** (fixé ici à $0,85$).

La formule d'itération combinant la matrice de transition $T$ et le saut aléatoire est :

$$\vec{p}^{(k+1)} = d \cdot (T \cdot \vec{p}^{(k)}) + \frac{1 - d}{N} \cdot \vec{1}$$

* **$d$** : représente la probabilité qu'un utilisateur continue sa navigation en suivant les liens
* **$T \cdot \vec{p}^{(k)}$** : gère la multiplication par la matrice creuse et la redistribution du poids des "impasses".
* **$(1 - d) / N$** : représente la probabilité pour un utilisateur de sauter vers n'importe quelle page du réseau de manière uniforme.

### Critère d'arrêt
L'algorithme s'arrête lorsque la variation relative entre deux itérations devient inférieure ou égale à un seuil $\epsilon$ :

$$\frac{\|T\vec{p}^{(k)} - \vec{p}^{(k)}\|_1}{\|\vec{p}^{(k)}\|_1} \leq \epsilon$$

ou plutôt, une fois le factor d'amortissement d implémenté :
 $$\frac{\|\vec{p}^{(k+1)} - \vec{p}^{(k)}\|_1}{\|\vec{p}^{(k)}\|_1} \leq \epsilon$$

Où la norme 1 est définie par $\|v\|_1 = \sum_{i=1}^{n} |v_i|$.

In [5]:
eps = 1e-9
k = 0
p_k = np.ones(nombre_pages) / nombre_pages
d = 0.85

while True:
    # L'opérateur @ déclenche la multiplication optimisée de SciPy
    p_kplus1 = d*(T @ p_k + p_k[impasses].sum()/nombre_pages) + (1 - d) / nombre_pages

    err = np.abs(p_kplus1 - p_k).sum() / np.abs(p_k).sum()
    p_k = p_kplus1
    k += 1

    if err <= eps:
        break

print(f"Convergence atteinte en {k} itérations avec une erreur de {err:.2e}.")
print(f"Somme de vérification du vecteur p : {p_k.sum():.4f}")

Convergence atteinte en 57 itérations avec une erreur de 8.63e-10.
Somme de vérification du vecteur p : 1.0000


## 5) Top pages

In [6]:
top_k = 20
idx = np.argsort(-p_k)[:top_k] # on trie par PageRank et on prend les k premiers par ordre DECROISSANT

pd.DataFrame({
    "rank": np.arange(1, top_k + 1),
    "node_id": idx + 1,
    "name": liste_pages["Name"].iloc[idx].to_numpy(),
    "pagerank": p_k[idx],
})


,rank,node_id,name,pagerank
0,1,112356,United States,0.002348
1,2,168241,United Kingdom,0.001282
2,3,138128,World War II,0.001051
3,4,60041,France,0.000990
4,5,184958,Latin,0.000853
5,6,138420,Germany,0.000847
6,7,151511,Canada,0.000729
7,8,49148,English language,0.000695
8,9,149853,China,0.000694
9,10,145591,India,0.000679


## 6) Recherche basique (titre contient le mot-clé)

On filtre par nom et trie par score PageRank

In [7]:
recherche = "New York"
top_k = 10

recherche = recherche.lower()
mask = liste_pages["Name"].str.lower().str.contains(recherche, na=False).to_numpy() # on filtre les pages contenant le mot clé
idx = np.where(mask)[0] # on récupère les indices des pages correspondantes
idx = idx[np.argsort(-p_k[idx])][:top_k] # on trie par PageRank et on prend les k premiers par ordre DECROISSANT

resultat = pd.DataFrame({
    "rank": np.arange(1, len(idx) + 1),
    "node_id": idx + 1,
    "name": liste_pages["Name"].iloc[idx].to_numpy(),
    "pagerank": p_k[idx],
})
resultat


,rank,node_id,name,pagerank
0,1,113505,New York City,0.000485
1,2,21033,New York,0.000346
2,3,113621,The New York Times,0.000302
3,4,125654,New York Stock Exchange,0.000053
4,5,113402,New York University,0.000050
5,6,96841,The New Yorker,0.000043
6,7,13273,"Buffalo, New York",0.000041
7,8,197152,New York Yankees,0.000038
8,9,21032,"Albany, New York",0.000028
9,10,13152,"Rochester, New York",0.000025
